# Notebook: Convert Model Results to Latex

This notebook is used to load the .json files with the model performance in order to convert them into a latex table for the paper

In [61]:
# Todo: Schauen, ob es die Metriken auch bei anderen Modellen gibt
# Todo: 1.000 <- Punkte einfügen
# Todo: Soll bei nur Real gehen
# Todo: Soll bei allen Tasks gehen
# Todo: bei f1 micro etc 3 nachkommastellen
# Schauen, dass es bei jedem Task "eval_f1_micro", "eval_f1_macro", "accuracy"

## Packages

In [62]:
import json
import numpy as np

## Settings / Constants

In [63]:
RESULTS_PATH_BASE = "../07 train models/results_json/results_"
LLMS = ["Llama3_70B", "GPT-3"]
# , "aspect_category_sentiment", "end_2_end_absa" ,"target_aspect_sentiment_detection"]
ABSA_TASKS = ["aspect_category", "aspect_category_sentiment",
              "end_2_end_absa", "target_aspect_sentiment_detection"]
SYNTH_COMBINATIONS = {
    "fixed": [
        {"real": 25, "synth": 475},
        {"real": 25, "synth": 975},
        {"real": 25, "synth": 1975}
    ],
    "random": [
        {"real": 500, "synth": 500},
        {"real": 500, "synth": 1000},
        {"real": 500, "synth": 1500}
    ]}

In [64]:
LLMS_ENCODED = {"GPT-3": "\\textbf{GPT-3.5-turbo}",
                "Llama2_70B": "\\textbf{Llama-2-70B}", "Llama3_70B": "\\textbf{Llama-3-70B}"}
ENCODE_CONDITION = {"fixed": "\\textbf{LRS\\textsubscript{25}}",
                    "random": "\\textbf{LRS\\textsubscript{500}}"}

N_METRICS = {"aspect_category": 6, "aspect_category_sentiment": 6,
             "end_2_end_absa": 6, "target_aspect_sentiment_detection": 5}

## Code

In [65]:
def round_number(num, decimal_places):
    formatted_num = "{:.{}f}".format(num, decimal_places)
    rounded_num_str = "{:.{}f}".format(float(formatted_num), decimal_places)
    return rounded_num_str

def add_thousand_dots(n_sample):
    if isinstance(n_sample, str):
        if '.' in n_sample:
            integer_part, decimal_part = n_sample.split('.')
            formatted_integer_part = "{:,}".format(int(integer_part))
            result = f"{formatted_integer_part}.{decimal_part}"
        else:
            result = "{:,}".format(int(n_sample))
    elif isinstance(n_sample, np.float64):
        result = "{:,}".format(round(n_sample, 1))
    else:
        result = n_sample
    
    return result

### Report Main Metrics

In [66]:
def get_metric(metric_name, results):
    main_metric = add_thousand_dots(round_number(results[metric_name]*100, 2))
    std_metric = round_number(np.std([res[metric_name] * 100 for res in results["single_split_results"]]), 2)
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [67]:
for absa_task in ABSA_TASKS:
    print(absa_task, ":\n\n", "-------#-----#-----#-------")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"

        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if absa_task != "target_aspect_sentiment_detection":
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {get_metric('eval_f1_micro', results)} & {get_metric('eval_f1_macro', results)} \\\\")
        else:
            print(
                f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {get_metric('eval_f1', results)} & {get_metric('eval_f1_macro', results)} \\\\")
    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(["fixed", "random"]):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                if absa_task != "target_aspect_sentiment_detection":
                    f1_metrics = f"{get_metric('eval_f1_micro', results)} & {get_metric('eval_f1_macro', results)}"
                else:
                    f1_metrics = f"{get_metric('eval_f1', results)} & {get_metric('eval_f1_macro', results)}"
                print(
                    f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {f1_metrics} \\\\")

                # print(llm_idx, fs_idx, freq_idx)
            if freq_idx == 2:
                print(
                    "\\arrayrulecolor{gray}\cline{2-"+str(6)+"}\\arrayrulecolor{black}")
            else:
                print("\\hline")
        print("\hline")
    print("\n\n")

aspect_category :

 -------#-----#-----#-------
\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 90.90\textsubscript{1.37} & 89.97\textsubscript{1.66} \\
 &  & 1,000 & 0 & 92.02\textsubscript{1.19} & 91.10\textsubscript{1.46} \\
 &  & 2,000 & 0 & 92.35\textsubscript{1.15} & 91.53\textsubscript{1.31} \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 81.33\textsubscript{0.36} & 80.37\textsubscript{1.30} \\
 &  & 25 & 975 & 80.76\textsubscript{2.03} & 80.18\textsubscript{1.65} \\
 &  & 25 & 1,975 & 80.65\textsubscript{1.77} & 80.20\textsubscript{1.11} \\
\arrayrulecolor{gray}\cline{2-6}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 89.88\textsubscript{1.42} & 88.77\textsubscript{1.72} \\
 &  & 500 & 1,000 & 88.77\textsubscript{1.06} & 87.15\textsubscript{1.10} \\
 &  & 500 & 1,500 & 88.49\textsubscript{1.61} & 87.11\textsubscript{1.55} \\
\arrayrulecolor{gray}\cli

### Report Metrics Fine-Grained Metrics

#### ACD

In [68]:
absa_task = "aspect_category"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
        for metric in ["f1"]:
            class_wise_metrics += f"{get_metric(f'eval_{metric}_{ac}', results)} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)

            if fs_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for ac in ["GENERAL-IMPRESSION", "FOOD", "SERVICE", "AMBIENCE", "PRICE"]:
                for metric in ["f1"]:
                    class_wise_metrics += f"{get_metric(f'eval_{metric}_{ac}', results)} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if fs_idx == 0 and freq_idx == 2:
               print(
                "\\arrayrulecolor{gray}\cline{2-9}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 85.23\textsubscript{3.71} &93.57\textsubscript{0.90} &93.03\textsubscript{1.84} &87.25\textsubscript{3.80} &90.78\textsubscript{3.57}  \\
 &  & 1,000 & 0 & 86.68\textsubscript{2.65} &94.71\textsubscript{0.92} &94.15\textsubscript{1.33} &88.21\textsubscript{2.87} &91.74\textsubscript{2.42}  \\
 &  & 2,000 & 0 & 87.41\textsubscript{2.77} &94.92\textsubscript{0.82} &94.41\textsubscript{1.10} &88.34\textsubscript{2.96} &92.56\textsubscript{3.00}  \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 71.22\textsubscript{2.92} &89.03\textsubscript{1.11} &82.79\textsubscript{2.49} &78.04\textsubscript{3.14} &80.75\textsubscript{7.14}  \\
 &  & 25 & 975 & 72.27\textsubscript{3.88} &87.38\textsubscript{1.97} &82.26\textsubscript{4.45} &79.12\textsubscript{1.97} &79.90\textsubscript{7.93}  \\
 &  & 25 & 1,975 & 71.15\textsubscript{2.53} &87.12\textsubscript{2.09} &83.

#### End-2-End ABSA

In [69]:
absa_task = "end_2_end_absa"
for n_real_idx, n_real in enumerate([500, 1000, 2000]):
    json_path = RESULTS_PATH_BASE + \
        f"only_real_real{n_real}_synth0_{absa_task}_random.json"
    with open(json_path, 'r') as json_file:
        results = json.load(json_file)

    if n_real_idx == 0:
        condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
    else:
        condition_print = " & "

    class_wise_metrics = ""
    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
        for metric in ["f1"]:
            class_wise_metrics += f"{get_metric(f'eval_{metric}_{polarity}', results)} &"
    print(
        f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 & {class_wise_metrics[:-1]} \\\\")
print("\\hline")
for llm_idx, llm in enumerate(LLMS):
    for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
        for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
            n_real = freq["real"]
            n_synth = freq["synth"]
            json_path = RESULTS_PATH_BASE + llm + \
                f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
            with open(json_path, 'r') as json_file:
                results = json.load(json_file)

            if fs_idx == 0 and freq_idx == 0:
                llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
            else:
                llm_print = ""

            if freq_idx == 0:
                condition_print = "\\multirow{3}{*}{" + \
                    ENCODE_CONDITION[few_shot_condition]+"}"
            else:
                condition_print = ""

            class_wise_metrics = ""
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["f1"]:
                    class_wise_metrics += f"{get_metric(f'eval_{metric}_{polarity}', results)} &"

            print(
                f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} & {class_wise_metrics[:-1]} \\\\")
            if fs_idx == 0 and freq_idx == 2:
               print(
                "\\arrayrulecolor{gray}\cline{2-7}\\arrayrulecolor{black}")

    print("\hline")

\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 82.11\textsubscript{2.60} &56.43\textsubscript{16.42} &71.67\textsubscript{2.53}  \\
 &  & 1,000 & 0 & 84.44\textsubscript{2.91} &63.94\textsubscript{7.56} &76.71\textsubscript{1.73}  \\
 &  & 2,000 & 0 & 85.03\textsubscript{3.83} &73.29\textsubscript{9.80} &78.18\textsubscript{4.12}  \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 60.42\textsubscript{3.22} &16.40\textsubscript{5.84} &56.69\textsubscript{3.00}  \\
 &  & 25 & 975 & 60.53\textsubscript{4.78} &19.88\textsubscript{11.85} &60.01\textsubscript{1.82}  \\
 &  & 25 & 1,975 & 62.27\textsubscript{5.12} &21.87\textsubscript{5.66} &61.41\textsubscript{3.97}  \\
\arrayrulecolor{gray}\cline{2-7}\arrayrulecolor{black}
 & \multirow{3}{*}{\textbf{LRS\textsubscript{500}}} & 500 & 500 & 79.48\textsubscript{1.91} &48.01\textsubscript{6.25} &73.10\textsubscript{2.50}  \\
 &  & 500 & 1,000 & 77.66\textsubscr

#### ACSA

In [70]:
def filter_str(values):
    return [value for value in values if isinstance(value, (int, float))]

def get_metric(metric, ac, polarity, results):
    metric_name = f'{metric}_{ac}-{polarity}'
    main_values = []

    for i in range(0, 6):
        if type(results["single_split_results"][i][f'eval_n_examples_{ac}-{polarity}']) != str:
            main_values.append(results["single_split_results"][i][metric_name] * 100)

    std_metric = round_number(np.std(main_values), 2)
    main_metric = add_thousand_dots(round_number(np.mean(main_values), 2))
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [71]:
absa_task = "aspect_category_sentiment"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1"]:
                    metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1"]:
                            metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 7 if ac_idx == 2 else 10
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 82.88\textsubscript{3.05} & 0.00\textsubscript{0.00} & 82.18\textsubscript{7.34} & 91.93\textsubscript{1.48} & 66.30\textsubscript{8.56} & 82.17\textsubscript{4.06} \\
 &  & 1,000 & 0 & 84.74\textsubscript{2.43} & 57.12\textsubscript{26.45} & 85.18\textsubscript{6.61} & 94.47\textsubscript{1.10} & 81.56\textsubscript{5.95} & 86.87\textsubscript{2.86} \\
 &  & 2,000 & 0 & 85.57\textsubscript{3.62} & 67.67\textsubscript{20.57} & 86.50\textsubscript{4.17} & 94.30\textsubscript{1.23} & 85.63\textsubscript{4.81} & 87.05\textsubscript{2.85} \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 70.83\textsubscript{4.27} & 25.83\textsubscript{15.92} & 71.38\textsubscript{9.87} & 65.02\textsubscript{4.84} & 37.55\textsubscript{13.64} & 65.85\textsubscript{5.55} \\
 &  & 25 & 975 & 72.16\textsubscript{2.29} & 18.14\textsubs

#### TASD

In [72]:
def filter_str(values):
    return [value for value in values if isinstance(value, (int, float))]


def get_metric(metric, ac, polarity, results):
    metric_name = f'{metric}_{ac}_{polarity}'
    
    main_values = []

    for i in range(0, 6):
        if results["single_split_results"][i][f'eval_n_examples_{ac}_{polarity}'] > 0:
            main_values.append(results["single_split_results"][i][metric_name] * 100)

    std_metric = round_number(np.std(main_values), 2)
    main_metric = add_thousand_dots(round_number(np.mean(main_values), 2))
    return main_metric + "\\textsubscript{" + std_metric + "}"

In [73]:
absa_task = "target_aspect_sentiment_detection"

idx = 0
for ac_idx, aspect_categories in enumerate([["GENERAL-IMPRESSION", "FOOD"], ["SERVICE", "AMBIENCE"], ["PRICE"]]):
    print("Table for:", aspect_categories, "\n\n")
    for n_real_idx, n_real in enumerate([500, 1000, 2000]):
        json_path = RESULTS_PATH_BASE + \
            f"only_real_real{n_real}_synth0_{absa_task}_random.json"
        with open(json_path, 'r') as json_file:
            results = json.load(json_file)

        if n_real_idx == 0:
            condition_print = "\\multirow{3}{*}{\\textbf{Real Examples}} & \\multirow{3}{*}{-}"
        else:
            condition_print = " & "

        condition_string = f"{condition_print} & {add_thousand_dots(str(n_real))} & 0 &"
        metrics_class_wise = ""
        for ac in aspect_categories:
            for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                for metric in ["eval_f1"]:
                    metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

        print(condition_string + metrics_class_wise[:-1] + "\\\\")

    print("\\hline")
    for llm_idx, llm in enumerate(LLMS):
        for fs_idx, few_shot_condition in enumerate(SYNTH_COMBINATIONS.keys()):
            for freq_idx, freq in enumerate(SYNTH_COMBINATIONS[few_shot_condition]):
                n_real = freq["real"]
                n_synth = freq["synth"]
                json_path = RESULTS_PATH_BASE + llm + \
                    f"_real{n_real}_synth{n_synth}_{absa_task}_{few_shot_condition}.json"
                with open(json_path, 'r') as json_file:
                    results = json.load(json_file)

                if fs_idx == 0 and freq_idx == 0:
                    llm_print = "\\multirow{6}{*}{"+LLMS_ENCODED[llm]+"}"
                else:
                    llm_print = ""

                if freq_idx == 0:
                    condition_print = "\\multirow{3}{*}{" + \
                        ENCODE_CONDITION[few_shot_condition]+"}"
                else:
                    condition_print = ""

                condition_string = f"{llm_print} & {condition_print} & {add_thousand_dots(str(n_real))} & {add_thousand_dots(str(n_synth))} &"
                metrics_class_wise = ""
                for ac in aspect_categories:
                    for polarity in ["POSITIVE", "NEUTRAL", "NEGATIVE"]:
                        for metric in ["eval_f1"]:
                            metrics_class_wise += f" {get_metric(metric, ac, polarity, results)} &"

                print(condition_string + metrics_class_wise[:-1] + "\\\\")
            
                n_col = 7 if ac_idx == 2 else 10
                if freq_idx == 2:
                   print("\\arrayrulecolor{gray}\cline{2-"+str(n_col)+"}\\arrayrulecolor{black}")
        print("\hline")
    idx += 1

Table for: ['GENERAL-IMPRESSION', 'FOOD'] 


\multirow{3}{*}{\textbf{Real Examples}} & \multirow{3}{*}{-} & 500 & 0 & 65.42\textsubscript{7.04} & 42.17\textsubscript{26.07} & 65.44\textsubscript{7.86} & 69.22\textsubscript{4.46} & 61.52\textsubscript{11.66} & 50.47\textsubscript{3.61} \\
 &  & 1,000 & 0 & 70.62\textsubscript{5.18} & 53.33\textsubscript{15.75} & 66.53\textsubscript{8.50} & 71.77\textsubscript{3.25} & 71.65\textsubscript{11.98} & 52.95\textsubscript{2.32} \\
 &  & 2,000 & 0 & 74.21\textsubscript{4.02} & 42.20\textsubscript{23.40} & 72.56\textsubscript{8.03} & 75.93\textsubscript{1.88} & 71.62\textsubscript{11.77} & 56.99\textsubscript{3.71} \\
\hline
\multirow{6}{*}{\textbf{Llama-3-70B}} & \multirow{3}{*}{\textbf{LRS\textsubscript{25}}} & 25 & 475 & 48.19\textsubscript{3.49} & 5.67\textsubscript{3.36} & 40.13\textsubscript{6.85} & 45.19\textsubscript{9.22} & 22.47\textsubscript{9.55} & 35.57\textsubscript{6.34} \\
 &  & 25 & 975 & 48.64\textsubscript{3.94} & 7.12\textsub

In [74]:
"The staff is very unfriendly, but it was delicious."[4:9]

'staff'

In [75]:
"It was very delicious, even the pizza was delicious"[32:37]

'pizza'